# Preparing datacards for CMS-Combine

This notebook reads into a text file containing yield information in the following format and turns them into pandas dataframes.
```
bin >> sig >> obs >> exp >> experr >> S/sqrtB >> dbkg
```


Each text file contains one model, for one specific signal region in one specific campaign. The dataframes from all signal regions and campaigns can be combined.

### Reading text files intdictionaryrd

In [1]:
import os, sys
import json
import numpy as np
import pandas as pd
import ROOT

print('Modules loaded.')

Welcome to JupyROOT 6.26/10
Modules loaded.


In [2]:
#Setting parameters

#Set which signal to probe
signame = "VLLD_mu"
#signame = "VLLD_ele"

#Mention which jobs to join
#jobs = ['hist_2017UL_sr_Jan07_mm']
jobs = ['hist_2017UL_sr_Jan07_em','hist_2017UL_sr_Jan07_me','hist_2017UL_sr_Jan07_mm',
        'hist_2018UL_sr_Jan07_em','hist_2018UL_sr_Jan07_me','hist_2018UL_sr_Jan07_mm']

#Find the output names:
outputname = f'datacards_{signame}_2017-18_combined'

sigdict = {
    'VLLD_ele': {
        'M100': {'mass': 100, 'xsec': 16.9,       'ngen': 110871, 'scale':10},
        'M200': {'mass': 200, 'xsec': 1.36,       'ngen': 73730 , 'scale':1},
        'M300': {'mass': 300, 'xsec': 0.291,      'ngen': 24753 , 'scale':1},
        'M400': {'mass': 400, 'xsec': 0.0907,     'ngen': 24491 , 'scale':1},
        'M600': {'mass': 600, 'xsec': 0.0149,     'ngen': 24611 , 'scale':1},
        'M800': {'mass': 800, 'xsec': 0.00347,    'ngen': 23680 , 'scale':1},
        'M1000': {'mass': 1000, 'xsec': 0.000971, 'ngen': 24286 , 'scale':1}
    },
    'VLLD_mu': {
        'M100': {'mass': 100, 'xsec': 16.9,    'ngen': 111926, 'scale':10},
        'M200': {'mass': 200, 'xsec': 1.36,    'ngen': 73908,  'scale':1},
        'M300': {'mass': 300, 'xsec': 0.291,   'ngen': 25022,  'scale':1},
        'M400': {'mass': 400, 'xsec': 0.0907,  'ngen': 24299 , 'scale':1},
        'M600': {'mass': 600, 'xsec': 0.0149,  'ngen': 24890,  'scale':1},
        'M800': {'mass': 800, 'xsec': 0.00347, 'ngen': 24763,  'scale':1}
    }
}

print('Global settings loaded.')

Global settings loaded.


In [3]:
def return_dict(jobname, signame):
    baseinputdir = '../StackMaker/signalyields/'
    indir = os.path.join(baseinputdir, jobname)
    df = {}

    for sample, subdict in sigdict.items():
        if sample not in signame: 
            continue

        if sample not in df: 
            df[sample] = {}

        for subsample, val in subdict.items():
            filename = f'yield_{sample}_{subsample}.txt'
            filepath = os.path.join(indir, filename)

            if not os.path.exists(filepath):
                print(f'\033[33mWarning: File not found: {filepath}\033[0m')
                continue

            try:
                # Read the file into a pandas DataFrame
                temp_df = pd.read_csv(
                    filepath,
                    sep=r'\s+',  # Split by one or more spaces
                    names=['bin', 'sig', 'obs', 'exp', 'experr', 'S/sqrtB', 'dbkg'],
                )

                # If subsample not present in df[sample], initialize numpy arrays for the columns
                if subsample not in df[sample]:
                    df[sample][subsample] = {
                        'bin': np.array([]),
                        'sig': np.array([]),
                        'obs': np.array([]),
                        'exp': np.array([]),
                        'experr': np.array([]),
                        'S/sqrtB': np.array([]),
                        'dbkg': np.array([]),
                    }

                # Append data from temp_df to the numpy arrays
                for col in temp_df.columns:
                    df[sample][subsample][col] = np.append(df[sample][subsample][col], temp_df[col].values)

            except Exception as e:
                print(f'\033[31mError loading file {filepath}: {e}\033[0m')

            #break #subsample
        #break  #sample

    return df

In [4]:
datadict = {}

for jobname in jobs:
    print(f'Reading job: {jobname}')
    
    dict_job = return_dict(jobname, signame)
    
    for sample, subdict in dict_job.items():
        if sample not in datadict:
            datadict[sample] = {}

        for subsample, columns in subdict.items():
            if subsample not in datadict[sample]:
                datadict[sample][subsample] = {
                    'bin': np.array([]),
                    'sig': np.array([]),
                    'obs': np.array([]),
                    'exp': np.array([]),
                    'experr': np.array([]),
                    'S/sqrtB': np.array([]),
                    'dbkg': np.array([]),
                }

            for col in columns: datadict[sample][subsample][col] = np.append(datadict[sample][subsample][col], columns[col])

print('Data collection complete!')

Reading job: hist_2017UL_sr_Jan07_em
Reading job: hist_2017UL_sr_Jan07_me
Reading job: hist_2017UL_sr_Jan07_mm
Reading job: hist_2018UL_sr_Jan07_em
Reading job: hist_2018UL_sr_Jan07_me
Reading job: hist_2018UL_sr_Jan07_mm
Data collection complete!


## Preparing datacard from the dictionary

In [5]:
def write_datacard(df, datacard):
    df = df.reset_index(drop=True)
    num_bins = len(df)  # Total number of bins
    
    if num_bins == 0:
        print(f'Warning: Zero bins detected! Skipping file {datacard}')
        return
    
    with open(datacard, 'w') as f:
        # Header information
        f.write(f"imax {num_bins}                          # number of channels\n")
        f.write(f"jmax 1                           # number of backgrounds\n")
        f.write(f"kmax {num_bins}                          # number of nuisance parameters\n")
        f.write("------------\n")
        
        # Bin section
        f.write(f"{'bin':<16}")
        line = ""
        for i in range(num_bins): line += f"bin{i + 1}\t"
        line = line[:-1]
        f.write(line + "\n")
        
        # Observation section
        f.write(f"{'observation':<16}")
        line = ""
        for i in range(num_bins): line += f"{int(df['obs'].iat[i])}\t"
        line = line[:-1]
        f.write(line + "\n")
        f.write("------------\n")

        # Bin-Bin section
        f.write(f"{'bin':<16}")
        line = ""
        for i in range(num_bins): line += f"bin{i + 1}\tbin{i + 1}\t"
        line = line[:-1]
        f.write(line + "\n")
        
        # Process section
        f.write(f"{'process':<16}")
        line = ""
        for i in range(num_bins): line += "sig\tbkg\t"
        line = line[:-1]
        f.write(line + "\n")

        # Process ID section:
        f.write(f"{'process':<16}")
        line = ""
        for i in range(num_bins): line += f"{-1*(i + 1)}\t{(i + 1)}\t"
        line = line[:-1]
        f.write(line + "\n")

        # Rate section
        f.write(f"{'rate':<16}")
        line = ""
        for i in range(num_bins): line += f"{df['sig'].iat[i]:.2f}\t{df['exp'].iat[i]:.2f}\t"
        line = line[:-1]
        f.write(line + "\n")
        f.write("------------\n")

        # Uncertainty:
        for i in range(num_bins):
            uncertainty_line = f"xs{i + 1:<6}lnN\t"
            values = []
            for j in range(num_bins):
                if j == i: # Diagonal element
                    values.append("-")  # Signal uncertainty
                    uncertainty_value = df['dbkg'].iat[i]
                    values.append(f"{uncertainty_value:.5f}") # Background uncertainty
                else:
                    values.append("-") # Signal uncertainty
                    values.append("-") # Background uncertainty
            uncertainty_line += "\t".join(values)
            f.write(uncertainty_line + "\n")
            
    print(f'Wrote file: {datacard}')

print('Function loaded.')

Function loaded.


In [6]:
outdir = f'datacards/{outputname}'
os.makedirs(outdir, exist_ok=True)

count = 0
for sample, subs in sigdict.items():
    if sample not in datadict: continue
    
    for subsample, val in subs.items():
        if subsample not in datadict[sample]: continue

        count+= 1
        sampleyield = datadict[sample][subsample]
        sample_df = pd.DataFrame(sampleyield)
        sample_df['bin'] = sample_df['bin'].astype(int)
        sample_df['obs'] = sample_df['obs'].astype(int)
        #sample_df = sample_df[sample_df['S/sqrtB']>0.0001] #dropping very small S/B
        sample_df_sorted = sample_df.sort_values(by='S/sqrtB', ascending=False)

        if count < 5:
            print(f"\nDataFrame for {sample}_{subsample}:")
            display(sample_df_sorted)

        datacard_name = f'datacard_{sample}_{subsample}.txt'
        datacard_path = os.path.join(outdir, datacard_name)
        write_datacard(sample_df_sorted, datacard_path)

print('Done!')


DataFrame for VLLD_mu_M100:


,bin,sig,obs,exp,experr,S/sqrtB,dbkg
36,2,334.36300,235,235.42900,5.937210,21.791600,1.02522
37,3,436.94800,557,557.58200,8.800850,18.504400,1.01578
15,2,209.40800,139,139.59400,3.702920,17.724000,1.02653
16,3,314.40300,354,354.68900,5.471020,16.694100,1.01542
23,3,598.14000,1566,1566.88000,33.808000,15.110700,1.02158
22,2,293.99600,448,448.45600,12.515900,13.883000,1.02791
2,3,421.47000,994,994.62200,17.804000,13.364000,1.01790
29,2,177.00200,216,216.61200,9.318080,12.026500,1.04302
1,2,194.18900,268,268.05900,8.110270,11.860700,1.03026
35,1,75.17070,45,45.78200,2.501700,11.109700,1.05464


Wrote file: datacards/datacards_VLLD_mu_2017-18_combined/datacard_VLLD_mu_M100.txt

DataFrame for VLLD_mu_M200:


,bin,sig,obs,exp,experr,S/sqrtB,dbkg
38,4,151.232000,369,369.48500,7.674830,7.867650,1.02077
17,4,105.150000,224,224.70000,4.748450,7.014680,1.02113
37,3,106.701000,557,557.58200,8.800850,4.518690,1.01578
16,3,75.916900,354,354.68900,5.471020,4.031020,1.01542
31,4,78.213600,514,514.73400,15.385500,3.447400,1.02989
10,4,55.028000,327,327.61400,10.197300,3.040200,1.03113
30,3,69.241800,645,645.14000,16.820300,2.726100,1.02607
9,3,49.681700,416,416.58400,11.110000,2.434140,1.02667
40,6,5.393000,6,6.67817,0.401555,2.086900,1.06013
39,5,12.890600,42,42.87380,1.686050,1.968680,1.03933


Wrote file: datacards/datacards_VLLD_mu_2017-18_combined/datacard_VLLD_mu_M200.txt

DataFrame for VLLD_mu_M300:


,bin,sig,obs,exp,experr,S/sqrtB,dbkg
38,4,47.312300,369,369.48500,7.674830,2.461360,1.02077
39,5,15.541200,42,42.87380,1.686050,2.373490,1.03933
17,4,33.836800,224,224.70000,4.748450,2.257290,1.02113
18,5,10.733200,25,25.67580,0.983865,2.118200,1.03832
31,4,36.695100,514,514.73400,15.385500,1.617400,1.02989
10,4,25.599200,327,327.61400,10.197300,1.414310,1.03113
40,6,2.091510,6,6.67817,0.401555,0.809341,1.06013
32,5,7.315850,83,83.65690,6.124240,0.799859,1.07321
11,5,5.270900,45,45.90320,3.412800,0.777971,1.07435
19,6,1.445670,4,4.31236,0.291840,0.696166,1.06768


Wrote file: datacards/datacards_VLLD_mu_2017-18_combined/datacard_VLLD_mu_M300.txt

DataFrame for VLLD_mu_M400:


,bin,sig,obs,exp,experr,S/sqrtB,dbkg
39,5,11.861800,42,42.87380,1.686050,1.811570,1.03933
18,5,8.864330,25,25.67580,0.983865,1.749380,1.03832
40,6,2.322310,6,6.67817,0.401555,0.898653,1.06013
19,6,1.715950,4,4.31236,0.291840,0.826316,1.06768
41,7,0.988197,2,2.13598,0.227016,0.676152,1.10628
11,5,4.280560,45,45.90320,3.412800,0.631799,1.07435
32,5,5.679430,83,83.65690,6.124240,0.620946,1.07321
20,7,0.750732,1,1.46235,0.169645,0.620811,1.11601
38,4,9.280010,369,369.48500,7.674830,0.482781,1.02077
17,4,6.593280,224,224.70000,4.748450,0.439845,1.02113


Wrote file: datacards/datacards_VLLD_mu_2017-18_combined/datacard_VLLD_mu_M400.txt
Wrote file: datacards/datacards_VLLD_mu_2017-18_combined/datacard_VLLD_mu_M600.txt
Wrote file: datacards/datacards_VLLD_mu_2017-18_combined/datacard_VLLD_mu_M800.txt
Done!
